# <div style="text-align:center; color:green">n-gram feature Extraction and Model Building using TF-IDF</div>

## importing neccesary libraries

In [1]:
import numpy as np
from nltk import word_tokenize
from pip._internal.utils.misc import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
import glob
import pandas as pd
%run stylometric_Features.ipynb import FeatureExtration
import seaborn as sns
import matplotlib as plt
import warnings
import string
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
%matplotlib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Using matplotlib backend: Qt5Agg


In [2]:
all_instances={

'hrudayashiva':["01","02","03","04","05","06","07","08","09",10,11,12],
    'ravibeliger':[13,14,15,16,17,18,19,20,21,22,23,24,25],
    'somashaker':[26,27,28,29,30,31,32,33,34,35,36],
    'Other author':[]
}
#from features_NormalApproach import FeatureExtration


stop_words_list = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"
data_folder = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train"
instance_by_author={}
for author,AllFiles_Per_author in all_instances.items():
    for i in AllFiles_Per_author:
        for name in glob.glob(f"C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train/instance{i}.txt"):
            with open(name, encoding='utf-8') as file:
                instance_by_author[author+str(i)]=file.read()

In [3]:
lemmatiser = WordNetLemmatizer()
stopword=[]
with open(stop_words_list, encoding='utf-8') as file:
    reader=file.read()
    stopword.append(reader)
def text_process(tex):
    nopunct = [char for char in tex if char not in string.punctuation]
    nopunct = ''.join(nopunct)
    return nopunct

In [4]:
def ngramFeatureExtraction():
    all_authors=[]
    text=[]
    for author, file in instance_by_author.items():
        all_authors.append(author)
        text.append(file)
    df=pd.DataFrame()
    df['author']=all_authors
#     df['author'][0:13]='hrudayashiva'
#     df['author'][13:25]='ravibeligeri'
#     df['author'][25:36]='somashaker'
    Y = pd.Series(df["author"])
    Y.replace(r'(^ra.*)','ravibeligeri',regex=True, inplace = True)
    Y.replace(r'(^hr.*)','hrudayashiva',regex=True, inplace = True)
    Y.replace(r'(^som.*)','somashaker',regex=True, inplace = True)            
    df['text']=text

    y = Y
    X = df['text']
    
    # 80-20 splitting the dataset (80%->Training and 20%->Validation)
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=1234)
    clean_data=[]
    
    #defining the bag-of-words transformer on the text-processed corpus # i.e., text_process() declared in II is executed...
    for i in X_train:
        clean_data.append(text_process(i))
    X_train = clean_data

    
    
    
    #bow_transformer=CountVectorizer(ngram_range=(2,3),min_df=5, analyzer=text_process).fit(X_train)
    word_vector=TfidfVectorizer(analyzer="word", ngram_range=(2,3),max_features=2000, binary=False).fit(X_train)
    word_vector1=TfidfVectorizer(analyzer="word", ngram_range=(3,4),max_features=2000, binary=False).fit(X_train)

    char_vector=TfidfVectorizer(analyzer="char", ngram_range=(2,3),max_features=2000, binary=False,min_df=3).fit(X_train)
    char_vector1=TfidfVectorizer(analyzer="char", ngram_range=(3,4),max_features=2000, binary=False,min_df=3).fit(X_train)
    bow_transformer=FeatureUnion([ ("chars",char_vector),("word",word_vector),("3_chars",char_vector1),("3_word",word_vector1)] )


    #bow_transformer =  TfidfVectorizer(min_df=3, analyzer=text_process).fit(X_train)
    #transforming into Bag-of-Words and hence textual data to numeric..
    feature_names = bow_transformer.get_feature_names()
    text_bow_train=bow_transformer.transform(X_train)#ONLY TRAINING DATA

    # transforming into Bag-of-Words and hence textual data to numeric..
    text_bow_test=bow_transformer.transform(X_test)#TEST DATA
    
    # instantiating the model with Multinomial Naive Bayes..
    model =SVC(kernel='linear')
    #model = model.fit(flatten_final.toarray(), y_train)

    #model = GaussianNB()
    #model= model.fit(flatten_final.toarray(), y_train)

    #model = RandomForestClassifier(max_depth=2, random_state=0)
    #model = model.fit(flatten_final.toarray(), y_train)


    # training the model...
    model = model.fit(text_bow_train, y_train)
    #import
    print("Training Accuracy :",model.score(text_bow_train, y_train))
    print("Testing Acuuracy  : ",model.score(text_bow_test, y_test))

    predictions = model.predict(text_bow_test)
    pred = pd.DataFrame()
    pred["text"] = X_test
    pred["author"]=predictions
    test_score = accuracy_score(y_test, predictions)
    print(classification_report(y_test, predictions))
#     print(pred,"\n")
#     print(df)
    return pred,df